# __DSPy Prompt Optimizers__: Optimizing instructions & fewshot examples for LM programs

Description of this notebook, what you'll learn, etc.

### 0] Setup

First, we'll __load in the cached requests__ for this tasks, so that we don't actually need to call any LMs for this notebook.

In [1]:
!rm -rf DSPy_optimizer_cache
!git clone https://huggingface.co/kopsahlong/DSPy_optimizer_cache
%cd DSPy_optimizer_cache/
# !git checkout master
%cd ..
import os
repo_clone_path = 'DSPy_optimizer_cache/cache' #TODO: update this cache to just contain the runs we need!!

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = repo_clone_path
os.environ["DSP_CACHEDIR"] = f"{repo_clone_path}/cachedir"


SyntaxError: unterminated string literal (detected at line 11) (1838760429.py, line 11)

In [3]:
!echo $HOME

/lfs/0/kristaoo/home


We will also specify the __prompt LM model__ (in this case GPT 3.5), the __task LM model__ (Llama 13B) and the retrieval model we'll be using for our task (a HotPotQA multihop retrieval task).

In [2]:
import os 
import dspy
import openai
import os

### NOTE: if you'd like to run this code without a cache, you can remove these lines to configure your OPEN AI key ###
# os.environ['OPENAI_API_KEY'] = "TODO: ADD YOUR OPEN AI KEY HERE"
# openai.api_key = os.environ.get('OPENAI_API_KEY')
# openai.api_base = "https://api.openai.com/v1"

prompt_model_name = "gpt-3.5-turbo-1106"
task_model_name = "meta-llama/Llama-2-13b-chat-hf"
colbert_v2_endpoint = "http://20.102.90.50:2017/wiki17_abstracts"

ports = [7140, 7141, 7142, 7143] #TODO: REMOVE THIS

prompt_model = dspy.OpenAI(model=prompt_model_name, max_tokens=150)
task_model = dspy.HFClientTGI(model=task_model_name, port=[7140, 7141, 7142, 7143], max_tokens=150)

colbertv2 = dspy.ColBERTv2(url=colbert_v2_endpoint)

dspy.settings.configure(rm=colbertv2, lm=task_model)

/lfs/0/kristaoo/miniconda3/envs/dspy_test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1] Define Task

Here, we'll define the program that we'd like to run, which is a multihop [...] (we can say that it was loosely inspired by a certain paper). We additionally load in the data, and define how we'd like to evaluate this task.

In [3]:
from dspy.evaluate import Evaluate
import re 
from dspy.datasets import HotPotQA

class ReturnRankedDocuments(dspy.Signature):
    """Given a question we are trying to answer and a list of passages, return a comma separated list of the numbers associated with each passage. These numbers should be ordered by helpfulness in answering the question, with most helpful passage number first, and the least helpful last."""
    question = dspy.InputField(desc="The question we're trying to answer.")
    context = dspy.InputField(desc="List of potentially related passages.")
    ranking = dspy.OutputField(desc="A comma separated list of numbers corresponding to passage indices, ranked in descending order by their helpfulness in answering our question.")

class RankingMultiHop(dspy.Module):
    def __init__(self, hops, num_passages_to_retrieve, max_passages_in_context):
        super().__init__()
        self.hops = hops
        self.num_passages_to_retrieve = num_passages_to_retrieve
        self.max_passages_in_context = max_passages_in_context
        self.retrieve = dspy.Retrieve(k = self.num_passages_to_retrieve)
        self.generate_query = dspy.ChainOfThought("context ,question->search_query")
        self.generate_answer = dspy.ChainOfThought("context ,question->answer")
        self.generate_ranking = dspy.ChainOfThought(ReturnRankedDocuments)
    
    def forward (self,question) :
        context = []
        full_context = []
        top_context = []
        max_passage_num = self.max_passages_in_context
        for hop in range(self.hops):
            # Get a new query
            query = self.generate_query(context = context, question = question).search_query
            # Get new passages
            context = self.retrieve(query).passages
            # Add these new passages to the previous top context 
            full_context = top_context + context
            # Get the most important indices, ranked
            most_important_indices =  self.generate_ranking(question=question, context=full_context).ranking
            indices = [int(num) for num in re.findall(r'\d+', most_important_indices)]

            if len(indices) < max_passage_num:
                indices = range(1,max_passage_num+1)

            valid_indices = [index-1 for index in indices if index-1 < len(context)]
            top_indices = sorted(valid_indices, key=lambda x: x)[:max_passage_num+1]
            most_important_context_list = [context[idx] for idx in top_indices]
            # Save the top context
            top_context = most_important_context_list

        return dspy.Prediction(context=context, answer=self.generate_answer(context = top_context , question = question).answer)

program = RankingMultiHop(hops=4, num_passages_to_retrieve=5, max_passages_in_context=5)

# Load and configure the datasets.
TRAIN_SIZE = 500
EVAL_SIZE = 500

hotpot_dataset = HotPotQA(train_seed=1, eval_seed=2023, test_size=0)
trainset = [x.with_inputs('question') for x in hotpot_dataset.train][:TRAIN_SIZE]
devset = [x.with_inputs('question') for x in hotpot_dataset.dev][:EVAL_SIZE]

# Set up metrics
NUM_THREADS = 10

metric = dspy.evaluate.answer_exact_match

# kwargs = dict(num_threads=NUM_THREADS, display_progress=True, display_table=None)
kwargs = dict(num_threads=NUM_THREADS, display_progress=True)
evaluate = Evaluate(devset=devset, metric=metric, **kwargs)

/lfs/0/kristaoo/miniconda3/envs/dspy_test/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


### 2] Baseline Evaluation
Now, we'll quickly evaluate our baseline program so that we can see how the performance using the Prompt Optimizer compares. We should see performance of about __16%__ on our trainset, and __21.4%__ on our devset.

In [4]:
baseline_train_score = evaluate(program,devset=trainset)
baseline_eval_score = evaluate(program, devset=devset)

#TODO: add in retrieval eval 

  0%|          | 0/500 [00:00<?, ?it/s]

/lfs/0/kristaoo/miniconda3/envs/dspy_test/lib/python3.10/site-packages/joblib/memory.py:655: JobLibCollisionWarning: Possible name collisions between functions 'colbertv2_get_request_v2_wrapped' (/lfs/0/kristaoo/dspy/dsp/modules/colbertv2.py:-1) and 'colbertv2_get_request_v2_wrapped' (/lfs/0/kristaoo/dspy/dsp/modules/colbertv2.py:52)
  return self._cached_call(args, kwargs)[0]
Average Metric: 80 / 500  (16.0): 100%|██████████| 500/500 [01:19<00:00,  6.26it/s]
/lfs/0/kristaoo/dspy/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 80 / 500  (16.0%)


Average Metric: 107 / 500  (21.4): 100%|██████████| 500/500 [01:20<00:00,  6.24it/s]


Average Metric: 107 / 500  (21.4%)


We can also inspect a trace from this program to see what a call using this program looks like in use. #TODO

### 3] Bayesian Optimization

#### 3a] Inspecting Pre-Optimized Program

First, because Bayesian Optimization can take a little while to run, let's load in a precompiled program to get a better understanding for what it looks like, and see how it performs.

#### 3a] Optimizing a Program from scratch

Now that we've seen what the Bayesian Prompt Optimizer can do, let's demonstrate how to train it. Note that because all of our calls to the LM are cached, this won't cost you anything, but it may take __~15 min__ to complete. [TODO: give a description of how the Bayesian Prompt Optimizer works]

In [5]:
from dspy.teleprompt import BayesianSignatureOptimizer

# Define hyperparameters:
N = 10 # The number of instructions and fewshot examples that we will generate and optimize over
trials = 30 # The number of optimization trials to be run (we will test out a new combination of instructions and fewshot examples in each trial) 
temperature = 1.0 # The temperature configured for generating new instructions

# Compile
eval_kwargs = dict(num_threads=16, display_progress=True, display_table=0)
teleprompter = BayesianSignatureOptimizer(prompt_model=prompt_model, task_model=task_model, metric=metric, n=N, init_temperature=temperature, verbose=False)
compiled_program = teleprompter.compile(program.deepcopy(), devset=trainset, optuna_trials_num=trials, max_bootstrapped_demos=1,max_labeled_demos=2, eval_kwargs=eval_kwargs)

DSPy_optimizer_cache/cache/compiler


  1%|          | 5/500 [00:00<00:08, 57.26it/s]


Bootstrapped 1 full traces after 6 examples in round 0.


  0%|          | 2/500 [00:00<00:09, 54.86it/s]


Bootstrapped 1 full traces after 3 examples in round 0.


...

Average Metric: 33 / 100  (33.0%)
Trial pruned.
Starting trial #28


Average Metric: 40 / 100  (40.0): 100%|██████████| 100/100 [00:07<00:00, 13.87it/s]
[I 2024-02-26 21:53:18,558] Trial 28 pruned. 


Average Metric: 40 / 100  (40.0%)
Trial pruned.
Starting trial #29


Average Metric: 34 / 100  (34.0): 100%|██████████| 100/100 [00:13<00:00,  7.28it/s]
[I 2024-02-26 21:53:32,494] Trial 29 pruned. 


Average Metric: 34 / 100  (34.0%)
Trial pruned.


## Evaluate

In [6]:
bayesian_train_score = evaluate(compiled_program,devset=trainset)
bayesian_eval_score = evaluate(compiled_program, devset=devset)

Average Metric: 215 / 500  (43.0): 100%|██████████| 500/500 [00:30<00:00, 16.32it/s]


Average Metric: 215 / 500  (43.0%)


Average Metric: 199 / 500  (39.8): 100%|██████████| 500/500 [01:21<00:00,  6.11it/s]


Average Metric: 199 / 500  (39.8%)
